In [ ]:
!pip install tenseal

In [ ]:
import tenseal as ts

context=ts.context(
        ts.SCHEME_TYPE.CKKS,
        poly_modulus_degree=8192,
        coeff_mod_bit_sizes=[60, 40, 40, 60]
    )
context.generate_galois_keys()
context.global_scale=2**40

In [ ]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

In [ ]:
X, y = make_classification(n_samples=1000, n_features=10, n_classes=2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
enc_X_train=[]
for i in X_train:
  p=ts.ckks_vector(context,i)
  enc_X_train.append(p)

In [ ]:
enc_y_train=[]
for i in y_train:
  p=ts.ckks_vector(context,[i])
  enc_y_train.append(p)

In [ ]:
def sigmoid(z):
  z=z.decrypt()
  z=round(z[0],7)
  x=1/(1+np.exp(-z))
  x=round(x,7)
  x=ts.ckks_vector(context,[x])
  return x

In [ ]:
def relinearlizing_vector(vec):
  vec_2=[]
  vec=vec.decrypt()
  for i in vec:
    p=round(i,7)
    vec_2.append(p)
  vec_2=ts.ckks_vector(context,vec_2)
  return vec_2

In [ ]:
def relinearlizing_scalar(sca):
  sca=sca.decrypt()
  sca=round(sca[0],7)
  sca=ts.ckks_vector(context,[sca])
  return sca

In [ ]:
n_samples,n_attributes=X_train.shape
n_samples,n_attributes

(800, 10)

In [ ]:
weights=np.zeros(n_attributes)
bias=0
lr=0.01
weights,bias

(array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), 0)

In [ ]:
weights=ts.ckks_vector(context,weights)
bias=ts.ckks_vector(context,[bias])

In [ ]:
for i in range(n_samples):
  xi=enc_X_train[i]
  yi=enc_y_train[i]
  lin_op=xi.dot(weights)
  lin_op=lin_op+bias
  y_pred=sigmoid(lin_op)
  db=y_pred-yi
  dw=db*xi
  dw=relinearlizing_vector(dw)
  db=relinearlizing_scalar(db)
  weights=relinearlizing_vector(weights)
  bias=relinearlizing_scalar(bias)
  weights=weights-(lr*dw)
  bias=bias-(lr*db)
  print(f'{i} loop is executed')

0 loop is executed
1 loop is executed
2 loop is executed
3 loop is executed
4 loop is executed
5 loop is executed
6 loop is executed
7 loop is executed
8 loop is executed
9 loop is executed
10 loop is executed
11 loop is executed
12 loop is executed
13 loop is executed
14 loop is executed
15 loop is executed
16 loop is executed
17 loop is executed
18 loop is executed
19 loop is executed
20 loop is executed
21 loop is executed
22 loop is executed
23 loop is executed
24 loop is executed
25 loop is executed
26 loop is executed
27 loop is executed
28 loop is executed
29 loop is executed
30 loop is executed
31 loop is executed
32 loop is executed
33 loop is executed
34 loop is executed
35 loop is executed
36 loop is executed
37 loop is executed
38 loop is executed
39 loop is executed
40 loop is executed
41 loop is executed
42 loop is executed
43 loop is executed
44 loop is executed
45 loop is executed
46 loop is executed
47 loop is executed
48 loop is executed
49 loop is executed
50 loop is

In [ ]:
weights

In [ ]:
weights.decrypt()

[-0.36015529776019706,
 0.048057853390585825,
 -0.5409007248772563,
 -0.0012974126867117339,
 -0.042012508435772616,
 -0.12965194092596788,
 1.1301058371318007,
 -0.01448903810719445,
 -0.3292970350845029,
 0.0349205063515082]

In [ ]:
bias.decrypt()

[0.03095578076495611]